In [1]:
!nvidia-smi


Fri Nov 14 19:44:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!apt-get -y install pciutils >/dev/null


In [3]:
!pip -q install langchain langchain-community neo4j pypdf pandas

import os, time, json, glob, hashlib, requests
import pandas as pd
from itertools import islice

from neo4j import GraphDatabase
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Neo4jVector


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
NEO4J_URI  = "bolt://18.206.150.250:7687"
NEO4J_USER = "neo4j"
NEO4J_PASS = "bangs-cliffs-lookout"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASS))

BASE = "http://127.0.0.1:11434"
LLM_MODEL = "llama3.1:8b"


In [5]:
!pkill -f ollama || true
!curl -fsSL https://ollama.com/install.sh | sh

import os, subprocess, time, requests
os.environ["OLLAMA_USE_CUDA"] = "1"

# start server
p = subprocess.Popen(["ollama", "serve"])

for _ in range(90):
    try:
        if requests.get("http://127.0.0.1:11434/api/version", timeout=2).ok:
            break
    except Exception:
        pass
    time.sleep(1)


^C
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [6]:
!ollama pull nomic-embed-text
!ollama pull llama3.1:8b

In [7]:
import re, glob, os
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

PDF_GLOB = "/content/pdf_listings2/listing_*.pdf"
pdf_paths = sorted(glob.glob(PDF_GLOB))
assert pdf_paths, "Нема PDF-и во /content/pdf_listings2/"

# 'Линк:' блок преку повеќе редови
link_block_re = re.compile(
    r"(Линк:?\s*)(https?://[^\s]+(?:\s+[^\s]+){0,15})",
    re.IGNORECASE
)

# fallback за сите http/https линкови
raw_url_re = re.compile(
    r"https?://[^\s]+",
    re.IGNORECASE
)

def clean_candidate_url(u: str) -> str:
    """
    Чисти URL кандидат од PDF текст.
    1. Сече кај зборови како 'Цена', 'Телефон', 'Објавено', 'Агенцијa' итн.
    2. Го ограничува на валидни URL карактери.
    3. Вади траилинг интерпункција.
    """
    CUT_TOKENS = [
        "цена", "Цена", "ЦЕНА",
        "телефон", "Телефон",
        "објавено", "Објавено",
        "оглас", "Оглас",
        "агенција", "Агенција",
    ]
    for tok in CUT_TOKENS:
        idx = u.find(tok)
        if idx != -1:
            u = u[:idx]

    # дозволени карактери во URL
    m = re.match(r"^(https?://[A-Za-z0-9\-._~/\d%?&=+#]+)", u.strip())
    if m:
        u = m.group(1)

    u = u.rstrip(").,;")
    return u.strip()

def pick_best_pazar3_link(all_urls):
    """
    Од листа кандидати избери најверојатно вистински линк кон оглас:
    - мора да содржи 'pazar3.mk' и '/oglas/'
    - ако има повеќе, земи го најдолгиот
    """
    pazar3 = [
        u for u in all_urls
        if "pazar3.mk" in u.lower() and "/oglas/" in u.lower()
    ]
    if not pazar3:
        return None
    pazar3.sort(key=len, reverse=True)
    return pazar3[0]

def extract_source_url_from_pdf_text(text: str) -> str | None:
    """
    1. Пробај преку 'Линк:' блок (full URL и ако е прекршен низ редови).
    2. Ако не успее, fallback: сите https://... во текстот.
    3. Исчисти ги кандидатите и избери најрелевантен pazar3.mk оглас линк.
    """
    candidates = []

    # 1) 'Линк:' блок
    m = link_block_re.search(text)
    if m:
        raw_block = m.group(2)
        pieces = raw_block.strip().split()
        glued = "".join(pieces)
        candidates.append(clean_candidate_url(glued))

    # 2) fallback: сите https... во flatten-нат текст
    flat_text = text.replace("\n", " ")
    for match in raw_url_re.findall(flat_text):
        candidates.append(clean_candidate_url(match))

    # 3) најдобар pazar3 линк
    best = pick_best_pazar3_link(candidates)
    return best

raw_docs = []
for p in pdf_paths:
    loader = PyPDFLoader(p)
    doc_id = os.path.splitext(os.path.basename(p))[0]

    for page in loader.load():
        text = page.page_content or ""
        meta = dict(page.metadata or {})
        meta["source_pdf"] = p
        meta["doc_id"] = doc_id

        # извлечен линк кон огласот
        found_url = extract_source_url_from_pdf_text(text)
        if found_url:
            meta["source_url"] = found_url

        # координати (ако постојат)
        m_c = re.search(r"([0-9]+\.[0-9]+)[,\s]+([0-9]+\.[0-9]+)", text)
        if m_c:
            meta["lat"] = float(m_c.group(1))
            meta["lon"] = float(m_c.group(2))

        raw_docs.append(Document(page_content=text, metadata=meta))

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
chunks = splitter.split_documents(raw_docs)
print("Total chunks:", len(chunks))


Total chunks: 210


In [8]:
with driver.session() as s:
    s.run("""
    CREATE VECTOR INDEX chunk_embed_p3 IF NOT EXISTS
    FOR (n:ChunkP3) ON (n.embedding)
    OPTIONS { indexConfig: { `vector.dimensions`: 768, `vector.similarity_function`: 'cosine' } }
    """)


In [9]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Neo4jVector
from itertools import islice

emb = OllamaEmbeddings(model="nomic-embed-text", base_url="http://127.0.0.1:11434")
_ = emb.embed_query("test")

store = Neo4jVector.from_documents(
    documents=[], embedding=emb,
    url=NEO4J_URI, username=NEO4J_USER, password=NEO4J_PASS,
    node_label="ChunkP3",
    text_node_property="text",
    embedding_node_property="embedding",
    index_name="chunk_embed_p3"
)

def batched(it, n):
    it = iter(it)
    while True:
        b = list(islice(it, n))
        if not b: break
        yield b

ing = 0
for b in batched(chunks, 512):
    store.add_documents(b)
    ing += len(b)
    print("Ingested:", ing)


/tmp/ipython-input-2120625780.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  emb = OllamaEmbeddings(model="nomic-embed-text", base_url="http://127.0.0.1:11434")


Ingested: 210


In [10]:
with driver.session() as s:
    s.run("MATCH (c:ChunkP3) WHERE c.chunkId IS NULL SET c.chunkId = elementId(c)")


In [11]:
from neo4j import GraphDatabase

with driver.session() as s:
    s.run("CREATE INDEX listing_uid   IF NOT EXISTS FOR (l:Listing)  ON (l.uid)")
    s.run("CREATE INDEX location_name IF NOT EXISTS FOR (n:Location) ON (n.name)")
    s.run("CREATE INDEX agent_name    IF NOT EXISTS FOR (a:Agent)    ON (a.name)")
    s.run("CREATE INDEX feature_name  IF NOT EXISTS FOR (f:Feature)  ON (f.name)")


In [12]:
with driver.session() as s:
    idx = s.run("SHOW INDEXES YIELD name, type, entityType, state RETURN name, type, entityType, state").data()
idx


[{'name': 'agent_name',
  'type': 'RANGE',
  'entityType': 'NODE',
  'state': 'ONLINE'},
 {'name': 'chunk_embed_p3',
  'type': 'VECTOR',
  'entityType': 'NODE',
  'state': 'ONLINE'},
 {'name': 'constraint_20c300d7',
  'type': 'RANGE',
  'entityType': 'NODE',
  'state': 'ONLINE'},
 {'name': 'feature_name',
  'type': 'RANGE',
  'entityType': 'NODE',
  'state': 'ONLINE'},
 {'name': 'index_343aff4e',
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'state': 'ONLINE'},
 {'name': 'index_f7700477',
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'state': 'ONLINE'},
 {'name': 'listing_uid',
  'type': 'RANGE',
  'entityType': 'NODE',
  'state': 'ONLINE'},
 {'name': 'location_name',
  'type': 'RANGE',
  'entityType': 'NODE',
  'state': 'ONLINE'}]

In [13]:
INSTR = """You are an IE engine. Extract a structured graph from REAL-ESTATE LISTING text.
Return STRICT JSON only:
{
  "listing": {
    "uid": "<string|null>",
    "title": "<string|null>",
    "listing_type": "<izdavanje|prodazba|unknown|null>",
    "property_type": "<stan|kukja|vikendica|posloven_prostor|drugo|null>",
    "price_eur": <number|null>,
    "price_period": "<per_month|per_m2|total|unknown|null>",
    "area_m2": <number|null>,
    "rooms": <number|null>,
    "features": ["<feature>", ...],
    "url": "<string|null>",
    "source": "<string|null>"
  },
  "location": { "name": "<string|null>", "lat": <number|null>, "lon": <number|null> },
  "agent": { "name": "<string|null>", "phone": "<string|null>" }
}
Rules:
- Detect listing_type (izdavanje vs prodazba) from text/URL:
  - if text contains 'издавање', 'се издава', 'изнајмување' → 'izdavanje'
  - if text contains 'продажба', 'се продава' → 'prodazba'
  - else 'unknown'
- Detect property_type:
  - if advert is for flat/apartment/стан → 'stan'
  - if for house/villa/куќа → 'kukja'
  - if weekend house → 'vikendica'
  - if office/business/посл. простор → 'posloven_prostor'
  - else 'drugo'
- Detect price period (per_month vs per_m2 vs total). Missing -> unknown.
- If a URL is present, set listing.url.
- If coordinates appear, set location.lat/lon numeric.
- 'features' should include attributes typically under sections like 'За живеалиштето',
  e.g. 'Балкон / Тераса', 'Паркинг простор / Гаража', 'Нова градба', 'Лифт', etc.
- Only JSON, no extra text."""

import os, re, json, time, hashlib, requests, subprocess

BASE = "http://127.0.0.1:11434"
LLM_MODEL_IE = "llama3.1:8b"

def _ensure_ollama_up(timeout=90):
    for _ in range(timeout):
        try:
            if requests.get(f"{BASE}/api/version", timeout=2).ok:
                return True
        except Exception:
            pass
        time.sleep(1)
    return False

def _restart_ollama():
    try: subprocess.run(["pkill","-f","ollama"], check=False)
    except Exception: pass
    os.environ.pop("OLLAMA_USE_CUDA", None)
    subprocess.Popen(["ollama","serve"])
    if not _ensure_ollama_up():
        raise RuntimeError("Ollama did not start")

# ---------- small regex heuristics ----------
_price_eur_re     = re.compile(r"(\d{1,3}(?:[.\s]\d{3})*|\d+)\s*(?:€|ЕУР|EUR)(?:\s*/\s*(?:месец|month|mo))?", re.I)
_price_per_m2_re  = re.compile(r"(\d{1,3}(?:[.\s]\d{3})*|\d+)\s*(?:€|ЕУР|EUR)\s*/\s*(?:m2|м2|м²|sqm)", re.I)
_area_m2_re       = re.compile(r"(\d{1,3}(?:[.\s]\d{3})*|\d+)\s*(?:m2|м2|м²|sqm)", re.I)
_rooms_re         = re.compile(r"\b(\d+)\s*(?:rooms?|соби|спални)\b", re.I)
_phone_re         = re.compile(r"(?:\+?\d[\d\s\-\/]{6,}\d)")

def _to_num(s):
    if s is None: return None
    return int(re.sub(r"[^\d]", "", s))

def heuristic_extract(text, meta):
    out = {
        "listing": {
            "uid": None,
            "title": None,
            "listing_type": None,
            "property_type": None,
            "price_eur": None,
            "price_period": None,
            "area_m2": None,
            "rooms": None,
            "features": [],
            "url": None,
            "source": None
        },
        "location": {"name": None, "lat": None, "lon": None},
        "agent": {"name": None, "phone": None}
    }
    t = " ".join((text or "").split())
    if meta:
        out["listing"]["url"] = meta.get("source_url") or None
        if meta.get("lat") is not None and meta.get("lon") is not None:
            out["location"]["lat"] = meta["lat"]; out["location"]["lon"] = meta["lon"]

    # цена / пер м2
    m = _price_per_m2_re.search(t)
    if m:
        out["listing"]["price_eur"] = _to_num(m.group(1))
        out["listing"]["price_period"] = "per_m2"
    else:
        m = _price_eur_re.search(t)
        if m:
            out["listing"]["price_eur"] = _to_num(m.group(1))
            out["listing"]["price_period"] = "per_month" if any(k in t.lower() for k in ["мес", "/ мес", "month", "/mo"]) else "total"

    # површина
    m = _area_m2_re.search(t)
    if m: out["listing"]["area_m2"] = _to_num(m.group(1))

    # соби
    m = _rooms_re.search(t)
    if m:
        try: out["listing"]["rooms"] = int(m.group(1))
        except: pass

    # телефон
    m = _phone_re.search(t)
    if m: out["agent"]["phone"] = m.group(0)

    # груба евруација
    score = sum(v is not None for v in [
        out["listing"]["price_eur"], out["listing"]["area_m2"],
        out["listing"]["price_period"], out["listing"]["url"]
    ])
    return out, score

def _json_mode_stream(prompt_obj):
    r = requests.post(f"{BASE}/api/generate", json={
        "model": LLM_MODEL_IE,
        "prompt": json.dumps(prompt_obj),
        "format": "json",
        "stream": True,
        "keep_alive": "30m",
        "options": {"temperature": 0.1, "num_predict": 180},
    }, stream=True, timeout=(10, 610))
    r.raise_for_status()
    buf=[]
    for line in r.iter_lines(decode_unicode=True):
        if not line: continue
        try:
            obj = json.loads(line)
            if "response" in obj: buf.append(obj["response"])
        except json.JSONDecodeError:
            continue
    return json.loads("".join(buf).strip())

def _json_mode_nostream(prompt_obj):
    r = requests.post(f"{BASE}/api/generate", json={
        "model": LLM_MODEL_IE,
        "prompt": json.dumps(prompt_obj),
        "format": "json",
        "stream": False,
        "keep_alive": "30m",
        "options": {"temperature": 0.1, "num_predict": 180},
    }, timeout=600)
    r.raise_for_status()
    return json.loads(r.json().get("response","").strip())

def _plain_mode_brace(prompt_text):
    r = requests.post(f"{BASE}/api/generate", json={
        "model": LLM_MODEL_IE,
        "prompt": prompt_text,
        "stream": False,
        "keep_alive": "30m",
        "options": {"temperature": 0.1, "num_predict": 200},
        "stop": ["\n\n", "```", "\nJSON", "\nText:"]
    }, timeout=600)
    r.raise_for_status()
    raw = r.json().get("response","").strip()
    b, e = raw.find("{"), raw.rfind("}")
    if b != -1 and e != -1:
        return json.loads(raw[b:e+1])
    raise ValueError("no JSON braces found")

def ollama_generate_json_ie(text, meta=None, max_retries=2):
    if not _ensure_ollama_up(): _restart_ollama()

    hints = []
    if meta:
        if meta.get("source_url"): hints.append({"hint":"url","value": meta["source_url"]})
        if meta.get("lat") is not None and meta.get("lon") is not None:
            hints.append({"hint":"coords","value": f'{meta["lat"]},{meta["lon"]}'})
    body = " ".join((text or "").split())[:2800]

    instr = {
        "instruction": "Extract structured real-estate info as JSON only.",
        "schema": {
            "listing": {
                "uid": "string|null",
                "title": "string|null",
                "listing_type": "izdavanje|prodazba|unknown|null",
                "property_type": "stan|kukja|vikendica|posloven_prostor|drugo|null",
                "price_eur": "number|null",
                "price_period": "per_month|per_m2|total|unknown|null",
                "area_m2": "number|null",
                "rooms": "number|null",
                "features": ["string"],
                "url": "string|null",
                "source": "string|null"
            },
            "location": { "name":"string|null", "lat":"number|null", "lon":"number|null" },
            "agent": { "name":"string|null", "phone":"string|null" }
        },
        "rules": [
            "Return ONLY valid JSON (no prose).",
            "If €/m² -> price_period=per_m2; if monthly -> per_month; else total/unknown.",
            "Detect listing_type and property_type from Macedonian words in the text/URL."
        ],
        "hints": hints
    }
    prompt_obj = {"schema":"listing_ie_v2","instr":instr,"text":body}

    # 0) heuristic fast-path
    hjson, score = heuristic_extract(text, meta)
    if score >= 2:
        return hjson

    # 1) JSON mode (stream)
    for _ in range(max_retries):
        try:
            return _json_mode_stream(prompt_obj)
        except (requests.exceptions.ReadTimeout, json.JSONDecodeError, requests.exceptions.ConnectionError):
            time.sleep(0.6)

    # 2) JSON mode (no stream)
    try:
        return _json_mode_nostream(prompt_obj)
    except Exception:
        pass

    # 3) Plain text with brace parse
    try:
        return _plain_mode_brace(f"{INSTR}\n\nTEXT:\n{body}\n\nJSON:")
    except Exception:
        pass

    # 4) Last-resort: fallback на heuristic JSON за да не заглави pipeline-от
    return {
        "listing": {
            "uid": None,
            "title": None,
            "listing_type": hjson["listing"].get("listing_type"),
            "property_type": hjson["listing"].get("property_type"),
            "price_eur": hjson["listing"]["price_eur"],
            "price_period": hjson["listing"]["price_period"],
            "area_m2": hjson["listing"]["area_m2"],
            "rooms": hjson["listing"]["rooms"],
            "features": hjson["listing"]["features"],
            "url": hjson["listing"]["url"],
            "source": meta.get("source_pdf") if meta else None
        },
        "location": hjson["location"],
        "agent": hjson["agent"]
    }

# ---- IE + UPSERT на партија ----
def extract_and_upsert_p3(items, batch_size=8):
    rows, done, total = [], 0, len(items)
    t0 = time.time()
    with driver.session() as s:
        for it in items:
            txt = (it["text"] or "").strip()
            if len(txt) < 40:
                s.run("MATCH (c:ChunkP3 {chunkId:$id}) SET c.extracted=true, c.extractedAt=timestamp()", id=it["chunkId"])
                done += 1
                continue

            data = ollama_generate_json_ie(txt, meta=it.get("meta"))

            meta_url = (it.get("meta") or {}).get("source_url")
            llm_url  = (data.get("listing") or {}).get("url")

            final_url = llm_url or meta_url

            listing = (data.get("listing") or {})
            location = (data.get("location") or {})
            agent = (data.get("agent") or {})

            rows.append({
                "chunkId": it["chunkId"],
                "hash": hashlib.sha1(txt.encode("utf-8")).hexdigest()[:16],
                "listing": {
                    "uid":           listing.get("uid"),
                    "title":         listing.get("title"),
                    "listing_type":  listing.get("listing_type"),
                    "property_type": listing.get("property_type"),
                    "price_eur":     listing.get("price_eur"),
                    "price_period":  listing.get("price_period"),
                    "area_m2":       listing.get("area_m2"),
                    "rooms":         listing.get("rooms"),
                    "features":      listing.get("features"),
                    "source":        listing.get("source"),
                    "url":           final_url,
                },
                "location": {
                    "name": location.get("name"),
                    "lat":  location.get("lat"),
                    "lon":  location.get("lon"),
                },
                "agent": {
                    "name":  agent.get("name"),
                    "phone": agent.get("phone"),
                },
                "chunk_url": meta_url
            })

            print("DEBUG ROW:",
                it["chunkId"],
                "final_url=", final_url,
                "meta_url=", meta_url,
                "llm_url=", listing.get("url"))

            if len(rows) >= batch_size:
                s.run(CYPHER_UPSERT, rows=rows)
                done += len(rows); rows=[]
                print(f"Upsert: {done}/{total}")

        if rows:
            s.run(CYPHER_UPSERT, rows=rows)
            done += len(rows)
            print(f"Upsert: {done}/{total} (final)")

    print("Graph upsert time:", round(time.time()-t0, 2), "s")


In [14]:
def fetch_chunks_p3(limit=None):
    q = """
    MATCH (c:ChunkP3)
    WHERE c.extracted IS NULL OR c.extracted = false
    RETURN c.chunkId AS id, c.text AS text, c AS node
    """
    if limit: q += " LIMIT $lim"
    with driver.session() as s:
        rows = s.run(q, lim=limit).data() if limit else s.run(q).data()
    out = []
    for r in rows:
        node_props = dict(r["node"])
        meta = {k: node_props.get(k) for k in ("source_url","source_pdf","lat","lon")}
        out.append({"chunkId": r["id"], "text": r["text"], "meta": meta})
    return out

CYPHER_UPSERT = """
UNWIND $rows AS r

// 1) најди го chunk-от и сетирај му ги основните проперти
MATCH (c:ChunkP3 {chunkId: r.chunkId})
SET c.url         = coalesce(c.url, r.chunk_url),
    c.extracted   = true,
    c.extractedAt = timestamp()

// 2) upsert Listing
MERGE (l:Listing {uid: coalesce(r.listing.uid, r.hash)})
SET l.title         = coalesce(r.listing.title, l.title),
    l.listing_type  = coalesce(r.listing.listing_type, l.listing_type),
    l.property_type = coalesce(r.listing.property_type, l.property_type),
    l.price_eur     = coalesce(r.listing.price_eur, l.price_eur),
    l.price_period  = coalesce(r.listing.price_period, l.price_period),
    l.area_m2       = coalesce(r.listing.area_m2, l.area_m2),
    l.rooms         = coalesce(r.listing.rooms, l.rooms),
    l.source        = coalesce(r.listing.source, l.source),
    l.url           = coalesce(r.listing.url, l.url, r.chunk_url)  // <- ова е клучно

WITH r, l, c
FOREACH (_ IN CASE WHEN r.location.name IS NULL THEN [] ELSE [1] END |
  MERGE (loc:Location {name: r.location.name})
  SET loc.lat = coalesce(r.location.lat, loc.lat),
      loc.lon = coalesce(r.location.lon, loc.lon)
  MERGE (l)-[:LOCATED_IN]->(loc)
)

WITH r, l, c
FOREACH (_ IN CASE WHEN r.agent.name IS NULL AND r.agent.phone IS NULL THEN [] ELSE [1] END |
  MERGE (a:Agent {name: coalesce(r.agent.name, r.agent.phone)})
  SET a.phone = coalesce(r.agent.phone, a.phone)
  MERGE (l)-[:LISTED_BY]->(a)
)

WITH r, l, c
FOREACH (f IN coalesce(r.listing.features, []) |
  MERGE (feat:Feature {name: f})
  MERGE (l)-[:HAS_FEATURE]->(feat)
)

// 3) поврзи listing ↔ chunk
MERGE (l)-[:MENTIONED_IN]->(c)
"""


In [16]:
assert _ensure_ollama_up(), "Ollama not up"
with driver.session() as s:
    print(s.run("MATCH (c:ChunkP3) WHERE c.extracted IS NULL OR c.extracted = false RETURN count(c) AS n").data())

sample = fetch_chunks_p3(limit=120)
extract_and_upsert_p3(sample, batch_size=8)


[{'n': 75}]
DEBUG ROW: 4:31a87c81-2f4b-48d2-92cc-cc57bcf130e9:120 final_url= https://www.pazar3.mk//oglas/zivealista/kukji-vili/prodazba/ohrid/ohrid-opstina/imperi meta_url= https://www.pazar3.mk//oglas/zivealista/kukji-vili/prodazba/ohrid/ohrid-opstina/imperi llm_url= https://www.pazar3.mk//oglas/zivealista/kukji-vili/prodazba/ohrid/ohrid-opstina/imperi
DEBUG ROW: 4:31a87c81-2f4b-48d2-92cc-cc57bcf130e9:121 final_url= https://www.pazar3.mk//oglas/zivealista/kukji-vili/prodazba/ohrid/ohrid-opstina/imperija-nedviznosti-kuka-od-190m2/1467274 meta_url= https://www.pazar3.mk//oglas/zivealista/kukji-vili/prodazba/ohrid/ohrid-opstina/imperija-nedviznosti-kuka-od-190m2/1467274 llm_url= https://www.pazar3.mk//oglas/zivealista/kukji-vili/prodazba/ohrid/ohrid-opstina/imperija-nedviznosti-kuka-od-190m2/1467274
DEBUG ROW: 4:31a87c81-2f4b-48d2-92cc-cc57bcf130e9:122 final_url= https://www.facebook.com/p/100063740252463/?_rdr meta_url= None llm_url= https://www.facebook.com/p/100063740252463/?_rdr
DEB

In [17]:
# Reinitialize на веќе постоечки граф
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Neo4jVector
from neo4j import GraphDatabase
import pandas as pd, requests, json

emb = OllamaEmbeddings(model="nomic-embed-text", base_url="http://127.0.0.1:11434")
store = Neo4jVector.from_existing_graph(
    embedding=emb,
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASS,
    index_name="chunk_embed",
    node_label="Chunk",
    text_node_properties=["text"],
    embedding_node_property="embedding",
)

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER,NEO4J_PASS))

In [18]:
from langchain_community.vectorstores import Neo4jVector
import pandas as pd
import json, requests

# re-attach на постоечкиот vector index
store_p3 = Neo4jVector.from_existing_graph(
    embedding=emb,
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASS,
    index_name="chunk_embed_p3",
    node_label="ChunkP3",
    text_node_properties=["text"],
    embedding_node_property="embedding",
)

def chunk_ids_for_hits_p3(hits):
    """
    Од листа на vector hits извлечи chunkId.
    Ако недостига во metadata, пробај да најдеш по text во графот (fallback).
    """
    ids = [h.metadata.get("chunkId") for h in hits if h.metadata.get("chunkId")]
    if len(ids) < len(hits):
        with driver.session() as s:
            for d in hits:
                if d.metadata.get("chunkId"):
                    continue
                row = s.run(
                    "MATCH (c:ChunkP3 {text:$t}) RETURN c.chunkId AS id LIMIT 1",
                    t=d.page_content
                ).single()
                if row and row["id"]:
                    ids.append(row["id"])
    return ids

def graph_expand_from_chunk_ids_p3(ids, top=10):
    """
    Од ChunkP3 → Listing + Location + Feature + Agent
    Враќа rows со деталите за огласот.
    """
    if not ids:
        return []

    q = """
    UNWIND $ids AS cid
    MATCH (l:Listing)-[:MENTIONED_IN]->(c:ChunkP3 {chunkId: cid})
    OPTIONAL MATCH (l)-[:LOCATED_IN]->(loc:Location)
    OPTIONAL MATCH (l)-[:HAS_FEATURE]->(f:Feature)
    OPTIONAL MATCH (l)-[:LISTED_BY]->(a:Agent)
    WITH l, loc, collect(DISTINCT f.name) AS feats, a
    RETURN
      l.uid           AS uid,
      l.title         AS title,
      l.price_eur     AS price_eur,
      l.price_period  AS price_period,
      l.area_m2       AS area_m2,
      l.rooms         AS rooms,
      l.listing_type  AS listing_type,
      l.property_type AS property_type,
      coalesce(loc.name, '?') AS location,
      loc.lat         AS lat,
      loc.lon         AS lon,
      feats           AS features,
      coalesce(a.name, '?') AS agent,
      a.phone         AS phone,
      l.url           AS url
    LIMIT $k
    """
    with driver.session() as s:
        return s.run(q, ids=ids, k=top).data()

# ќе го користиме истиот модел како за останатото
LLM_MODEL_ANS = LLM_MODEL

def ollama_short(prompt, num_predict=220):
    """
    Краток одговор од LLM (стриминг).
    """
    r = requests.post(
        f"{BASE}/api/generate",
        json={
            "model": LLM_MODEL_ANS,
            "prompt": prompt,
            "stream": True,
            "options": {"temperature": 0.2, "num_predict": num_predict},
        },
        stream=True,
        timeout=(10, 610),
    )
    r.raise_for_status()
    out = []
    for line in r.iter_lines(decode_unicode=True):
        if not line:
            continue
        try:
            obj = json.loads(line)
            if "response" in obj:
                out.append(obj["response"])
        except json.JSONDecodeError:
            continue
    return "".join(out)

def ask_hybrid_p3(qtext, k_vec=12, k_listings=8):
    """
    Hybrid RAG:
      1) Vector search по ChunkP3
      2) Graph expand до Listing/Location/Feature/Agent
      3) LLM одговор базиран само на табелата + линкови
    """
    # 1) Vector RAG
    hits = store_p3.similarity_search(qtext, k=k_vec)
    if not hits:
        return "Нема релевантни парчиња."

    # 2) Graph RAG (expand)
    cids = chunk_ids_for_hits_p3(hits)
    rows = graph_expand_from_chunk_ids_p3(cids, top=k_listings)

    # Ако графот сè уште е празен (нема Listing), користи само текст
    if not rows:
        ctx = "\n\n".join(
            [f"[{i+1}] {d.page_content}" for i, d in enumerate(hits, 1)]
        )
        prompt = (
            "Одговори во детали ИСКЛУЧИВО од следниов текст на македонски:\n\n"
            f"{ctx[:4000]}"
        )
        return ollama_short(prompt)

    # 3) Табела од огласи
    df = pd.DataFrame(rows)
    table_txt = df.to_string(index=False)[:3500]

    # 4) Генерација листа на цитати (URL-ови)
    cites = []
    for i, d in enumerate(hits[:8], 1):
        cid = d.metadata.get("chunkId")
        link = None

        if cid:
            with driver.session() as s:
                row = s.run("""
                    MATCH (c:ChunkP3 {chunkId:$cid})
                    OPTIONAL MATCH (l:Listing)-[:MENTIONED_IN]->(c)
                    RETURN coalesce(l.url, c.url) AS linkOut
                    LIMIT 1
                """, cid=cid).single()
            if row:
                link = row["linkOut"]

        # fallback ако ни графот нема url
        if not link:
            link = (
                d.metadata.get("source_url")
                or d.metadata.get("source_pdf")
            )

        cites.append(f"[{i}] {link if link else '(no link)'}")

    prompt = f"""Одговори на македонски, концизно но детално, ИСКЛУЧИВО од табелата и цитатите.

ПРАШАЊЕ:
{qtext}

ТАБЕЛА (огласи):
{table_txt}

ЦИТАТИ (линкови кон огласите):
{chr(10).join(cites)}

Правила:
- Прво објасни накратко што најде (колку огласи се релевантни).
- Потоа наведи 2–5 најрелевантни опции со цена, локација, тип (стан/куќа), издавање/продажба.
- Ако има features (Балкон / Тераса, Паркинг, Нова градба итн.), искористи ги.
- На крај предложи кои огласи најмногу одговараат на барањето и зошто.
"""

    return ollama_short(prompt)


In [19]:
print(ask_hybrid_p3("Дај ми линк за стан за изнајмување во Аеродром"))


Најдено е 1 релевантен оглас за стан за изнајмување во Аеродром.

**Релевантни опции:**

1. **Оглас:** https://www.pazar3.mk//oglas/zivealista/stanovi/izdavanje/skopje/aerodrom/se-izdava-stan-aerodrom-82m2/8092167
	* Цена: 1700 €/месец
	* Локација: Аеродром
	* Тип: Стан
	* Издавање: Изнајмување
	* Features: Балкон, Паркинг
2. **Оглас:** https://www.pazar3.mk//oglas/zivealista/kukji-vili/izdavanje/skopje/aerodrom/hit-samostojna-ponova-kukja-vo-lisice/557458


In [ ]:
import pandas as pd

def hybrid_listing_urls(qtext, k_vec=12, k_listings=15):
    """
    Користи embedding пребарување + граф за да ги најде огласите
    и враќа ЛИСТА ОД УНИКАТНИ URL-ови (без LLM).
    """
    # 1) Vector search по chunks
    hits = store_p3.similarity_search(qtext, k=k_vec)
    if not hits:
        return []

    # 2) Собери chunkId за погодените парчиња
    cids = chunk_ids_for_hits_p3(hits)
    if not cids:
        return []

    # 3) Прошири преку графот во Listing јазли
    rows = graph_expand_from_chunk_ids_p3(cids, top=k_listings)
    if not rows:
        return []

    # 4) Извади уникатни URL-ови
    urls = []
    for r in rows:
        u = r.get("url")
        if u and u not in urls:
            urls.append(u)
    return urls


def hybrid_listing_urls_with_details(qtext, k_vec=12, k_listings=15):
    """
    Враќа (DataFrame, [url1, url2, ...]) за да имаш и детали и линкови.
    """
    hits = store_p3.similarity_search(qtext, k=k_vec)
    if not hits:
        return pd.DataFrame(), []

    cids = chunk_ids_for_hits_p3(hits)
    rows = graph_expand_from_chunk_ids_p3(cids, top=k_listings)
    if not rows:
        return pd.DataFrame(), []

    df = pd.DataFrame(rows)
    urls = []
    for r in rows:
        u = r.get("url")
        if u and u not in urls:
            urls.append(u)
    return df, urls

def ask_hybrid_links_p3(qtext, k_vec=12, k_listings=8):
    """
    Користи ист pipeline како ask_hybrid_p3, но LLM мора да врати
    само URL-ови (по еден во ред).
    """
    hits = store_p3.similarity_search(qtext, k=k_vec)
    if not hits:
        return "Нема релевантни парчиња."

    cids = chunk_ids_for_hits_p3(hits)
    rows = graph_expand_from_chunk_ids_p3(cids, top=k_listings)
    if not rows:
        return "Нема погодени огласи во графот."

    df = pd.DataFrame(rows)
    table_txt = df.to_string(index=False)[:3000]

    cites = []
    for i, d in enumerate(hits[:8], 1):
        cid = d.metadata.get("chunkId")
        link = None
        if cid:
            with driver.session() as s:
                row = s.run("""
                    MATCH (c:ChunkP3 {chunkId:$cid})
                    OPTIONAL MATCH (l:Listing)-[:MENTIONED_IN]->(c)
                    RETURN coalesce(l.url, c.url) AS linkOut
                    LIMIT 1
                """, cid=cid).single()
            if row:
                link = row["linkOut"]
        if not link:
            link = d.metadata.get("source_url") or d.metadata.get("source_pdf")
        cites.append(f"[{i}] {link if link else '(no link)'}")

    prompt = f"""Извади од табелата и од цитатите САМО URL адреси на огласите
(на pazar3.mk или други релевантни страни) кои одговараат на барањето.

ФОРМАТ НА ОДГОВОР:
Врати само URL-ови, ПО ЕДЕН ВО РЕД, без дополнителен текст, без објаснување.

ПРАШАЊЕ:
{qtext}

ТАБЕЛА:
{table_txt}

ЦИТАТИ:
{chr(10).join(cites)}
"""

    return ollama_short(prompt, num_predict=200)


In [ ]:
print(ask_hybrid_links_p3(
    "Дади ми огласи за станови за ИЗНАЈМУВАЊЕ во Центар "
    "со Балкон / Тераса и Паркинг простор / Гаража."
))



https://www.pazar3.mk//oglas/zivealista/kukji-vili/prodazba/ohrid/ohrid-opstina/agenczi
https://www.pazar3.mk//oglas/zivealista/kukji-vili/prodazba/skopje/aerodrom/hit-samostojna-ponova-kukja-vo-lisice/5574586
https://www.pazar3.mk//oglas/zivealista/kukji-vili/izdavanje/skopje/karpos/nova-kukja-vo-karposh-zhdanec/7100697
https://www.pazar3.mk//oglas/zivealista/stanovi/prodazba/skopje/aerodrom/se-prodava-stan-aerodrom-82m/8092167
https://www.pazar3.mk//oglas/zivealista/k
